In [1]:
from embit import bip39, bip32
import os, hashlib

In [2]:
mnemonic = bip39.mnemonic_from_bytes(hashlib.sha256(b"random stuff").digest())
mnemonic

'carry decide decline search grant chest demand cram crane creek confirm ill limb text bar almost donor club prefer hand future hair endless proof'

In [3]:
seed = bip39.mnemonic_to_seed(mnemonic, '')
seed.hex()

'f7718b42e345f991289e3db39ca0f64ed8c25a2481d8bbd31cd972b96ea902dbca30c2c00fe631f279371343c1cbd0e71d6c1cde944ab62b26d058cd9b8e79bc'

In [4]:
root = bip32.HDKey.from_seed(seed)
root.to_base58()

'xprv9s21ZrQH143K3vWs32LN8d3nPPWywCw6DRtPZdb6qpZHQtuT1iDogKWcqT9EpRuEn5ivoTe7B8iNUza4A8527crvTKGhexMaG54494QjEaF'

In [5]:
root.serialize()[4:].hex()

'000000000000000000bb4dc15b22e4c399c49458bc45581d64133796673f3c4adfbda24a12d58bf6f500a971004c8f2e179bb31a2af499c39bc180e94bac65eee46ba0439632f14cc060'

In [6]:
from applets.core import get_connection, SecureAppletBase

class BlindOracle(SecureAppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CE01", connection)

    def set_seed(self, seed):
        return self.secure_request(b'\x10\x02'+seed)

    def derive_seed(self, password=b''):
        return self.secure_request(b'\x10\x04'+password)

    def get_xpub(self, derivation=b''):
        return self.secure_request(b'\x11\x01'+derivation)
    
    def sign(self, msg):
        return self.secure_request(b'\x11\x02'+msg)

In [7]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

In [8]:
app = BlindOracle(connection)
app.select()

b''

In [9]:
app.establish_secure_channel()
app.is_secure_channel_open

True

In [10]:
app.set_seed(seed)

b''

In [11]:
app.derive_seed()

b''

In [12]:
r = app.get_xpub()
r.hex()

'000000000000000000bb4dc15b22e4c399c49458bc45581d64133796673f3c4adfbda24a12d58bf6f503db1a6665e5bccaaa71543c18549d87bbac7e9a10d80f0ef0c9924c8c083ec352'

In [13]:
root.to_public().serialize()[4:] == r

True

In [14]:
der = bip32.parse_path("m/44h/1h/0h/0/1")
der

[2147483692, 2147483649, 2147483648, 0, 1]

In [15]:
derivation = b"".join([d.to_bytes(4,'big') for d in der])
derivation.hex()

'8000002c80000001800000000000000000000001'

In [17]:
res = app.get_xpub(derivation)
res.hex()

'05d609b81500000001d04ace55922a1b6f284ca52e0ffbfa6493d346b63886efe68ab256f14d9c7d95024dd8ed7b990476dbe7252e73f9eeb324b3020be841b0e5e1174262fb1f621e71'

In [18]:
root.derive(der).to_public().serialize()[4:] == res

True

In [19]:
msg = b'5'*32

In [20]:
pub = root.derive(der).to_public().key

In [21]:
sig = app.sign(msg)
sig.hex()

'3045022100a1fe432130e972d4768a2368c01a0fc0bf4292c7226972ba0f2d72cd03c5dcb802201b52fab947c4815170c12ad3dc2655979a18fb189a527fa7c88266aa6dd12b08'

In [22]:
from embit import ec
s = ec.Signature.parse(sig)

In [23]:
pub.verify(s, msg)

True

In [24]:
connection.disconnect()